In [2]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

###  **What are Outputparsers in Langchain**
Output parsers in LangChain are components that convert the text responses from language models into structured data formats that your application can more easily use. They act as the bridge between the unstructured text output from LLMs and structured data formats needed for downstream processing.

##### Why Use Output Parsers?
- Converts free-form text into structured data 
- Helps extract reliable & clean responses 
- Useful for API calls, automation, and databases



### Types of Output Parsers in LangChain

- StringOutputParser: The simplest parser that just returns the raw string output.
- JSONOutputParser: Parses LLM responses into JSON objects.
- PydanticOutputParser: Converts outputs into Pydantic models, providing schema validation.
- StructuredOutputParser: Parses outputs into predefined structured formats.

### StringOutputParser

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
string_parser = StrOutputParser()

chain = prompt | model | string_parser

result = chain.invoke({"topic": "programming"})
print(result)  # Prints the raw text response from the LLM

Why do programmers prefer dark mode? Because the light attracts bugs!


### JSONOutputParser

In [4]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Define the structure we want
json_parser = JsonOutputParser()

# Create a prompt that asks for JSON format
prompt = ChatPromptTemplate.from_template("""
Generate information about a fictional character.
Return it as a JSON with the following keys: name, age, background, skills.
{format_instructions}
""")

# Format the prompt with instructions
format_instructions = json_parser.get_format_instructions()
print(format_instructions)

model = ChatOpenAI(temperature=0)
# Create the chain but pass the format_instructions when invoking
chain = prompt | model | json_parser

# Pass the format_instructions in the input dictionary
result = chain.invoke({"format_instructions": format_instructions})
print(result)  # Prints a Python dictionary

Return a JSON object.
{'name': 'Elena Rodriguez', 'age': 28, 'background': 'Elena grew up in a small town in Mexico where she learned traditional healing practices from her grandmother. She later moved to the city to study medicine and now works as a doctor in a busy hospital.', 'skills': ['Herbal medicine', 'Acupuncture', 'Surgery']}


### PydanticOutputParser

In [5]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# Define our data model
class Character(BaseModel):
    name: str = Field(description="The character's name")
    age: int = Field(description="The character's age")
    skills: List[str] = Field(description="The character's abilities")
    background: str = Field(description="Short biography of the character")

# Create the parser
parser = PydanticOutputParser(pydantic_object=Character)

# Set up the prompt
prompt = ChatPromptTemplate.from_template("""
Create a detailed fictional character profile.
{format_instructions}
""")

# Get formatting instructions
format_instructions = parser.get_format_instructions()
formatted_prompt = prompt.format(format_instructions=format_instructions)

model = ChatOpenAI(temperature=0.7)
chain = prompt | model | parser

result = chain.invoke({'format_instructions': format_instructions})
print(result)  # Prints a Character object that can be accessed with dot notation
print(result.name)  # Access specific fields
print(result.skills)

name='Aria Nightshade' age=28 skills=['Stealth', 'Archery', 'Lockpicking', 'Disguise'] background='Aria Nightshade is a skilled rogue who grew up on the streets of the bustling city of Valoria. She learned to fend for herself from a young age, using her quick wit and nimble fingers to survive. As she got older, she honed her skills in stealth and archery, becoming a sought-after thief for high-stakes heists. Despite her shady profession, Aria has a strong sense of justice and often uses her talents to help those in need. She is known for her daring escapades and ability to outsmart even the most cunning adversaries.'
Aria Nightshade
['Stealth', 'Archery', 'Lockpicking', 'Disguise']


### StructuredOutputParser

In [6]:
# from langchain_core.output_parsers import ResponseSchema , StructuredOutputParser
from langchain.output_parsers.structured import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Define the response schemas
response_schemas = [
    ResponseSchema(name="title", description="The title of the article"),
    ResponseSchema(name="summary", description="A brief summary of the article's content"),
    ResponseSchema(name="key_points", description="3-5 key points from the article"),
    ResponseSchema(name="audience", description="The intended audience for this article")
]

# Create the parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Create the prompt
prompt = ChatPromptTemplate.from_template("""
Write an article outline about {topic}.
{format_instructions}
""")

# Format the prompt with instructions
format_instructions = parser.get_format_instructions()
formatted_prompt = prompt.format(format_instructions=format_instructions, topic="artificial intelligence")

model = ChatOpenAI(temperature=0)
chain = prompt | model | parser

result = chain.invoke({"topic": "artificial intelligence" , "format_instructions": format_instructions})
print(result)  # Prints a dictionary with keys matching the schema names
print(result["title"])  # Access specific fields by dictionary keys
print(result["key_points"])

{'title': 'Exploring the World of Artificial Intelligence', 'summary': 'This article delves into the fascinating realm of artificial intelligence, discussing its history, current applications, and future potential.', 'key_points': '1. The origins of artificial intelligence date back to the 1950s. 2. AI is used in various industries such as healthcare, finance, and transportation. 3. Machine learning and deep learning are key components of AI technology. 4. The ethical implications of AI development are a growing concern. 5. The future of AI holds promise for advancements in robotics, autonomous vehicles, and more.', 'audience': 'Technology enthusiasts, students, and professionals interested in learning more about artificial intelligence.'}
Exploring the World of Artificial Intelligence
1. The origins of artificial intelligence date back to the 1950s. 2. AI is used in various industries such as healthcare, finance, and transportation. 3. Machine learning and deep learning are key compon